In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2di33yst'

In [3]:
wandb.init(id='3jf8wk1i', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 56.4 s, sys: 2.02 s, total: 58.4 s
Wall time: 58.4 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [9]:
for n, p in m.named_parameters():
    """if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False"""
        
    print(n, p.requires_grad)

hidden_init True
cell_init True
backbone.conv_stem.weight True
backbone.bn1.weight True
backbone.bn1.bias True
backbone.blocks.0.0.conv_dw.weight True
backbone.blocks.0.0.bn1.weight True
backbone.blocks.0.0.bn1.bias True
backbone.blocks.0.0.se.conv_reduce.weight True
backbone.blocks.0.0.se.conv_reduce.bias True
backbone.blocks.0.0.se.conv_expand.weight True
backbone.blocks.0.0.se.conv_expand.bias True
backbone.blocks.0.0.conv_pw.weight True
backbone.blocks.0.0.bn2.weight True
backbone.blocks.0.0.bn2.bias True
backbone.blocks.0.1.conv_dw.weight True
backbone.blocks.0.1.bn1.weight True
backbone.blocks.0.1.bn1.bias True
backbone.blocks.0.1.se.conv_reduce.weight True
backbone.blocks.0.1.se.conv_reduce.bias True
backbone.blocks.0.1.se.conv_expand.weight True
backbone.blocks.0.1.se.conv_expand.bias True
backbone.blocks.0.1.conv_pw.weight True
backbone.blocks.0.1.bn2.weight True
backbone.blocks.0.1.bn2.bias True
backbone.blocks.1.0.conv_pw.weight True
backbone.blocks.1.0.bn1.weight True
backb

In [10]:
model_stem = "8.18"
START_E = 41

In [11]:
n_epochs = 1000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 46 else 3e-4 if e < 55 else 1e-4
    #opt.param_groups[0]['lr'] = 1e-4 #6e-4 if e < 1 else 3e-4 if e < 2 else 1e-4
    #opt.param_groups[0]['lr'] = 1e-4
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 41

{'trn_control_loss': 0.01581389, 'trn_headings_loss': 0.00063017, 'trn_curvatures_loss': 0.00279755, 'consistency losses/trn_steer_cost': 0.00391287, 'consistency losses/trn_te_loss': 18.33482116, 'aux losses/trn_pitch_loss': 0.15289068, 'aux losses/trn_yaw_loss': 0.25556803, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.3984375, 'consistency losses/trn_torque_loss': 579.56360302, 'consistency losses/trn_torque_delta_loss': 11.2082254, 'logistical/obs_consumed_per_second': 233.7007874, 'logistical/obs_generated_per_second': 102.37007874, 'logistical/data_consumption_ratio': 2.28194458, 'logistical/manual_train_pause': 7.874e-05, 'logistical/max_param': 16.16862297, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 232.484375, 'logistical/obs_generated_per_second': 99.078125, 'logistical/data_consumption_ratio': 2.34524522, 'logistical/manual_train_pause': 0.00039062, 'trn_control_loss': 0.02720147, 'trn_headings_loss': 0.00108948, 'trn_curvatures_los

{'logistical/obs_consumed_per_second': 230.8046875, 'logistical/obs_generated_per_second': 105.875, 'logistical/data_consumption_ratio': 2.17787128, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02196115, 'trn_headings_loss': 0.00097382, 'trn_curvatures_loss': 0.00467774, 'consistency losses/trn_steer_cost': 0.00692801, 'consistency losses/trn_te_loss': 18.58684673, 'aux losses/trn_pitch_loss': 0.15756273, 'aux losses/trn_yaw_loss': 0.29877806, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.4140625, 'consistency losses/trn_torque_loss': 601.76775145, 'consistency losses/trn_torque_delta_loss': 8.67015071, 'logistical/max_param': 16.17409706, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 234.9453125, 'logistical/obs_generated_per_second': 105.25, 'logistical/data_consumption_ratio': 2.23542029, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01822388, 'trn_headings_loss': 0.00079164, 'trn_curvatures_loss': 0.00317159, 'consistency l

{'logistical/obs_consumed_per_second': 238.0625, 'logistical/obs_generated_per_second': 99.640625, 'logistical/data_consumption_ratio': 2.38621596, 'logistical/manual_train_pause': 0.0003125, 'trn_control_loss': 0.0142495, 'trn_headings_loss': 0.00059027, 'trn_curvatures_loss': 0.00249756, 'consistency losses/trn_steer_cost': 0.0046498, 'consistency losses/trn_te_loss': 18.21186168, 'aux losses/trn_pitch_loss': 0.1561203, 'aux losses/trn_yaw_loss': 0.27881241, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.3515625, 'consistency losses/trn_torque_loss': 666.11785126, 'consistency losses/trn_torque_delta_loss': 9.91148916, 'logistical/max_param': 16.15900612, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 234.796875, 'logistical/obs_generated_per_second': 103.7109375, 'logistical/data_consumption_ratio': 2.26433399, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0154531, 'trn_headings_loss': 0.00065613, 'trn_curvatures_loss': 0.00252882, 'consiste

{'logistical/obs_consumed_per_second': 238.703125, 'logistical/obs_generated_per_second': 107.8828125, 'logistical/data_consumption_ratio': 2.21978456, 'logistical/manual_train_pause': 0.00015625, 'trn_control_loss': 0.02068675, 'trn_headings_loss': 0.00080949, 'trn_curvatures_loss': 0.00313962, 'consistency losses/trn_steer_cost': 0.00601278, 'consistency losses/trn_te_loss': 19.40030244, 'aux losses/trn_pitch_loss': 0.12754941, 'aux losses/trn_yaw_loss': 0.3569603, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.5859375, 'consistency losses/trn_torque_loss': 724.03137183, 'consistency losses/trn_torque_delta_loss': 10.91964217, 'logistical/max_param': 16.16428947, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 227.125, 'logistical/obs_generated_per_second': 97.6875, 'logistical/data_consumption_ratio': 2.32881825, 'logistical/manual_train_pause': 0.000625, 'trn_control_loss': 0.02138096, 'trn_headings_loss': 0.00084848, 'trn_curvatures_loss': 0.00442339, 'c

{'logistical/obs_consumed_per_second': 234.953125, 'logistical/obs_generated_per_second': 106.484375, 'logistical/data_consumption_ratio': 2.20566668, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02394819, 'trn_headings_loss': 0.00094157, 'trn_curvatures_loss': 0.00422363, 'consistency losses/trn_steer_cost': 0.00753738, 'consistency losses/trn_te_loss': 17.37960073, 'aux losses/trn_pitch_loss': 0.19266367, 'aux losses/trn_yaw_loss': 0.43294239, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.4453125, 'consistency losses/trn_torque_loss': 637.30238348, 'consistency losses/trn_torque_delta_loss': 8.68978858, 'logistical/max_param': 16.17683983, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 230.0703125, 'logistical/obs_generated_per_second': 100.734375, 'logistical/data_consumption_ratio': 2.2843711, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.03558165, 'trn_headings_loss': 0.00141732, 'trn_curvatures_loss': 0.00618188, 'c

...

{'logistical/obs_consumed_per_second': 165.0, 'logistical/obs_generated_per_second': 103.0, 'logistical/data_consumption_ratio': 1.60553957, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 59

{'trn_control_loss': 0.01107991, 'trn_headings_loss': 0.00042159, 'trn_curvatures_loss': 0.00158387, 'consistency losses/trn_steer_cost': 0.00361566, 'consistency losses/trn_te_loss': 19.1260227, 'aux losses/trn_pitch_loss': 0.09158373, 'aux losses/trn_yaw_loss': 0.17848873, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.25, 'consistency losses/trn_torque_loss': 493.28798517, 'logistical/obs_consumed_per_second': 234.77165354, 'logistical/obs_generated_per_second': 102.69291339, 'logistical/data_consumption_ratio': 2.28812552, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_delta_loss': 4.93179864, 'logistical/max_param': 16.19094086, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 231.8046875, 'logistical/obs_generated_per_second': 106.22656

{'logistical/obs_consumed_per_second': 233.3203125, 'logistical/obs_generated_per_second': 100.7734375, 'logistical/data_consumption_ratio': 2.31716897, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.01053023, 'trn_headings_loss': 0.00041777, 'trn_curvatures_loss': 0.00194172, 'consistency losses/trn_steer_cost': 0.00476399, 'consistency losses/trn_te_loss': 18.52239013, 'aux losses/trn_pitch_loss': 0.13721561, 'aux losses/trn_yaw_loss': 0.17920732, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.28125, 'consistency losses/trn_torque_loss': 780.52860582, 'consistency losses/trn_torque_delta_loss': 7.53269218, 'logistical/max_param': 16.1872654, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 231.96875, 'logistical/obs_generated_per_second': 105.0703125, 'logistical/data_consumption_ratio': 2.20686369, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01211119, 'trn_headings_loss': 0.0004556, 'trn_curvatures_loss': 0.0021375, 'cons

{'logistical/obs_consumed_per_second': 241.078125, 'logistical/obs_generated_per_second': 105.9609375, 'logistical/data_consumption_ratio': 2.27948195, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00999656, 'trn_headings_loss': 0.00037829, 'trn_curvatures_loss': 0.00157463, 'consistency losses/trn_steer_cost': 0.0035483, 'consistency losses/trn_te_loss': 18.84230968, 'aux losses/trn_pitch_loss': 0.083004, 'aux losses/trn_yaw_loss': 0.16334224, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.1875, 'consistency losses/trn_torque_loss': 710.38113049, 'consistency losses/trn_torque_delta_loss': 6.43834555, 'logistical/max_param': 16.1865406, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 227.484375, 'logistical/obs_generated_per_second': 104.421875, 'logistical/data_consumption_ratio': 2.18211909, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.01082841, 'trn_headings_loss': 0.00041214, 'trn_curvatures_loss': 0.00151361, 'consist

{'logistical/obs_consumed_per_second': 233.078125, 'logistical/obs_generated_per_second': 103.734375, 'logistical/data_consumption_ratio': 2.24711835, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01155365, 'trn_headings_loss': 0.0004666, 'trn_curvatures_loss': 0.00199599, 'consistency losses/trn_steer_cost': 0.00438735, 'consistency losses/trn_te_loss': 17.44652161, 'aux losses/trn_pitch_loss': 0.10998082, 'aux losses/trn_yaw_loss': 0.1651783, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.1953125, 'consistency losses/trn_torque_loss': 727.9854514, 'consistency losses/trn_torque_delta_loss': 6.60071484, 'logistical/max_param': 16.18753815, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.65625, 'logistical/obs_generated_per_second': 102.9375, 'logistical/data_consumption_ratio': 2.28902518, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01082551, 'trn_headings_loss': 0.00048859, 'trn_curvatures_loss': 0.00195478, 'consistency lo

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e63.torch")

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
